In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import re

In [ ]:
from selenium import webdriver
driver = webdriver.Edge()

### WEB SCRAPPING 
### Using IMDb url.

In [ ]:
driver.get('https://www.imdb.com/list/ls050782187/')

In [ ]:
driver.execute_script('return document.body.scrollHeight')

In [ ]:
driver

In [ ]:
old_height = driver.execute_script('return document.body.scrollHeight')
import time
while True:

    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    time.sleep(4)
    new_height = driver.execute_script('return document.body.scrollHeight')

    if new_height == old_height:
        break
    old_height = new_height

In [ ]:
html = driver.page_source

In [ ]:
html

In [ ]:
soup = BeautifulSoup(html)
soup

### Html tags , for loops.


In [ ]:
Movie = [Movie.get_text(strip = True) for Movie in soup.find_all('h3',class_ = 'ipc-title__text')]
# Movie

Movie = Movie[:-3]

In [ ]:
Imdb = [i.get_text(strip = True) for i in soup('span', class_='ipc-rating-star--rating')]
# Imdb

voting = [v.get_text(strip = True) for v in soup('span',class_= 'ipc-rating-star--voteCount')]
# voting

In [ ]:
year = []
genre = []
duration = []
meta = []
director =[]


for i in soup.find_all('div', class_='sc-5bc66c50-5 hVarDB dli-title-metadata'):
    regex = re.findall('\d{4}', i.text) 
    if len(regex) > 0: 
        year.append(regex[0])


for i in soup.find_all('div',class_='sc-5bc66c50-5 hVarDB dli-title-metadata'):
    #print(i.text)
    regex = re.findall('\d+h\s\d+m(.*)', i.text)
    if len(regex)>0:
        genre.append(regex[0])
    else:
        genre.append(np.nan)


for i in soup.find_all('div',class_='sc-5bc66c50-5 hVarDB dli-title-metadata'):
    #print(i.text)
    regex = re.findall(r'\d{4}(\d+h\s\d+m)', i.text)
    if len(regex)>0:
        duration.append(regex[0])
    else:
        duration.append(np.nan)


for i in soup.find_all('span',class_='sc-5bc66c50-1 dzqfDt'):
    #print(i.text)
    regex = re.findall(r'Rate(\d+)M', i.text)
    #print(regex)
    if len(regex)>0:
        meta.append(regex[0])
    else:
        meta.append(np.nan)


for i in soup.find_all("div",class_="sc-54004b59-1 eeStly"):
    a = i.find("a",class_='ipc-link ipc-link--base dli-director-item')
    if a :
        director.append(a.text)

### Checking for length

In [ ]:
print(len(Imdb))
print(len(voting))
print(len(year))
print(len(genre))
print(len(duration))
print(len(meta))
print(len(director))
print(len(Movie))

In [ ]:
driver.close()

## Dataset Conversion

In [ ]:
data = pd.DataFrame({
    'Movie_Title' : Movie,'Year' : year,'Duration' : duration,
    'Imdb_Rating' : Imdb,'Certificate' :genre,
    'Voting' : voting,'Metascore_Rating' : meta,'Directors' : director})

In [ ]:
data

### DataFrame To CSV file

In [ ]:
data.to_csv('IMDB-H.csv',index = False)